# Cell Centered Stencil using unstructured LDC

In [ ]:
from meshpy import triangle, geometry
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
from src.mesher import *
from src.environment import *
from src.dynamics import *
from src.refiner import MeshRefiner
from tqdm import tqdm

## Building Geometry

In [ ]:
builder = geometry.GeometryBuilder()

Add Circle

In [ ]:
# points,facets,_,facet_markers = geometry.make_circle(.5,(0.0,0.0),marker = 1)
# builder.add_geometry(points,facets,facet_markers=facet_markers)

Add Box

In [ ]:
points,facets,_,_ = geometry.make_box((0, 0), (2.0, 2.0))#,subdivisions=(200, 200))
builder.add_geometry(points,facets, facet_markers=[1,1,2,1])

## Setting Mesh Info

In [ ]:
info = triangle.MeshInfo()
builder.set(info)
#info.set_holes([(0.0, 0.0)]) # Sets center circle as a hole

In [ ]:
for facet, marker in zip(builder.facets, builder.facet_markers):
    match marker:
        case 1:
            color = 'blue'
        case 2:
            color = 'green'
        case 3:
            color = 'orange'
        case 4:
            color = 'purple'
    plt.plot(np.asarray(builder.points)[facet, 0], np.asarray(builder.points)[facet, 1], color=color, linewidth=0.5)
    plt.text(np.mean(np.asarray(builder.points)[facet, 0]), np.mean(np.asarray(builder.points)[facet, 1]), str(marker), fontsize=8, ha='center')
plt.show()

Meshing

In [ ]:
mesh = triangle.build(info,min_angle=33.0,
                      max_volume=.0001,
                      generate_faces=True,
                      generate_neighbor_lists=True,
                      attributes=True,
                      volume_constraints=True)

In [ ]:
np.array(mesh.elements).shape

In [ ]:
improver = MeshRefiner(mesh)
improver.show_mesh_quality()
mesh = improver.improve(aspect_thresh=3.0,
                        skew_thresh=30.0,
                        max_volume=0.0001,
                        move_fraction=.25,
                        max_iter=40)
improver.show_mesh_quality()

## Plotting

In [ ]:
mesh_points = np.array(mesh.points)
mesh_tris = np.array(mesh.elements)
mesh_attr = np.array(mesh.point_markers)

import matplotlib.pyplot as plt

plt.triplot(mesh_points[:, 0], mesh_points[:, 1], mesh_tris)
plt.xlabel("x")
plt.ylabel("y")
#
n = np.size(mesh_attr)
inner_nodes = [i for i in range(n) if mesh_attr[i] == 1]
outer_nodes = [i for i in range(n) if mesh_attr[i] == 3]
plt.plot(mesh_points[inner_nodes, 0], mesh_points[inner_nodes, 1], "ro")
plt.plot(mesh_points[outer_nodes, 0], mesh_points[outer_nodes, 1], "go")
plt.axis("equal")
plt.axis([0,.2,0,.2])
plt.show()

In [ ]:
import sys
sys.path.append('..')
from src.mesher import *
from src.environment import *
from src.dynamics import *
mesher = Mesher()
mesher.import_meshpy(mesh)

In [ ]:
mesher.calc_mesh_properties()

In [ ]:
mesher.verify_stencil_geometry()

In [ ]:
xy = np.asarray(mesher.points)
x = xy[:,0]
y = xy[:,1]
tri = np.asarray(mesher.cells)

# Find the node at (0, 0)
corner_node = np.where((mesher.points[:, 0] == 0) & (mesher.points[:, 1] == 0))[0][0]
corner_coords = mesher.points[corner_node]
# Find all nodes with both coordinates greater than corner_coords
candidates = np.where((mesher.points[:, 0] > corner_coords[0]) & (mesher.points[:, 1] > corner_coords[1]))[0]
if len(candidates) > 0:
    # Find the candidate with minimal distance to (0,0)
    dists = np.linalg.norm(mesher.points[candidates] - corner_coords, axis=1)
    diagonal_node = candidates[np.argmin(dists)]
    print(f"Diagonal node index: {diagonal_node}, coordinates: {mesher.points[diagonal_node]}")
else:
    print("No diagonal node found.")
    diagonal_node = None

if diagonal_node is not None:
    # Find all cells that include the diagonal node
    cells_on_diagonal_node = [i for i, cell in enumerate(mesher.cells) if diagonal_node in cell]
    print(f"Cell indices on diagonal node {diagonal_node}: {cells_on_diagonal_node}")
else:
    cells_on_diagonal_node = []

for cell_index in cells_on_diagonal_node:
    face_indices = mesher.cell_face_indices[cell_index]
    face_centers = mesher.face_centers[face_indices]
    face_normals = mesher.face_normals[face_indices]
    stencil_normals = mesher.stencil_norms[face_indices]
    for i, (fc, nrm, snrm, dist) in enumerate(zip(face_centers, face_normals, stencil_normals, mesher.cc_stencil_dist[face_indices])):
        # Plot points along the stencil normal direction
        pt_plus = fc + snrm * dist[1]  # outward from cell center to face (stencil direction)
        pt_minus = fc - snrm * dist[0] # inward toward cell center (stencil direction)
        print(f"Face {i}: Center={fc}, Face Normal={nrm}, Stencil Normal={snrm}, Distances={dist}, Point Plus={pt_plus}, Point Minus={pt_minus}")
        plt.plot([pt_plus[0]], [pt_plus[1]], marker='s', color='orange', markersize=6, label='Outward Stencil' if i == 0 else "")
        plt.plot([pt_minus[0]], [pt_minus[1]], marker='s', color='purple', markersize=6, label='Inward Stencil' if i == 0 else "")

    plt.triplot(x, y, tri[cell_index:cell_index + 1], color="g")
    plt.plot(mesher.cell_centers[cell_index, 0], mesher.cell_centers[cell_index, 1], marker='o', color='red', markersize=5, label='Cell Center')
    plt.plot(face_centers[:, 0], face_centers[:, 1], marker='x', color='green', markersize=5, label='Face Center', linestyle='None')
    # Plot both face normals (blue) and stencil normals (magenta)
    plt.quiver(face_centers[:, 0], face_centers[:, 1], face_normals[:, 0], face_normals[:, 1], color='blue', angles='xy', scale_units='xy', scale=200, label='Face Normals')
    plt.quiver(face_centers[:, 0], face_centers[:, 1], stencil_normals[:, 0], stencil_normals[:, 1], color='magenta', angles='xy', scale_units='xy', scale=200, label='Stencil Normals')
plt.axis('equal')
plt.axis([-.01, .02, -.01, .02])
plt.show()

In [ ]:
# Given physical parameters
Re = 100            # Reynolds number
nu = 0.1     # physical kinematic viscosity
L = 100             # domain length (physical units)
dx = 1           # spatial step (physical units)
dt = .1            # time step (physical units)

# Speed of sound squared for D2Q9 lattice
c_s_sq = 1.0 / 3.0

# Convert to lattice units
U_lattice = Re * nu / L

# Compute relaxation time tau
Tau = nu / c_s_sq + 0.5

# Print results
print(f"Lid velocity (lattice units): {U_lattice:.5f}")
print(f"Relaxation time tau: {Tau:.5f}")
U_lid = U_lattice
dynamics = D2Q9(tau=Tau,delta_t = dt)

In [ ]:
cells,faces,nodes = mesher.to_env(dynamics,flux_method="cc_alt_upwind")
nodes = mesher.set_vel_node(nodes, marker=1, velocity=jnp.array([0.0, 0.0]))
nodes = mesher.set_vel_node(nodes,marker=2, velocity=jnp.array([U_lid, 0.0]))

In [ ]:
env = Environment(cells,faces,nodes)
env.init()

In [ ]:
for i in tqdm(range(10)):
    env = env.step()

In [ ]:
env = env.step()

In [ ]:
xy = np.array(mesher.cell_centers)
x = xy[:,0]
y = xy[:,1]
vel = env.cells.vel 
mag = np.sqrt(np.sum(vel**2,axis=-1))
plt.quiver(x,y,vel[:,0],vel[:,1],mag,scale=1)
xy = np.asarray(mesher.points)
x = xy[:,0]
y = xy[:,1]
tri = np.asarray(mesher.cells)
plt.triplot(x,y,tri)
plt.axis([1.9,2,1.9,2])
#plt.axis([-1,-.9,-1,-.9])
plt.colorbar()
plt.show()

In [ ]:
xy = np.array(mesher.cell_centers)
x = xy[:,0]
y = xy[:,1]
vel = env.cells.vel
mag = np.sqrt(np.sum(vel**2,axis=-1))
plt.scatter(x, y, c=mag, cmap='viridis', vmin=0, vmax=jnp.max(mag), s=1, marker='o')
plt.axis('equal')
plt.show()

In [ ]:
from scipy.interpolate import LinearNDInterpolator
interp = LinearNDInterpolator(xy, vel)

In [ ]:
x = np.linspace(0, 2, 1000)
y = np.linspace(0, 2, 1000)
x,y = np.meshgrid(x, y)
vel = interp(x,y)
plt.streamplot(x, y, vel[:,:,0], vel[:,:,1],density=1)
plt.axis('equal')
plt.show()

In [ ]:
x = np.linspace(0, 2, 75)
y = np.linspace(0, 2, 75)
x,y = np.meshgrid(x, y)
vel = interp(x,y)
plt.quiver(x,y,vel[:,:,0],vel[:,:,1],scale=1)
plt.axis('equal')
plt.show()

In [ ]:
from scipy.io import loadmat
ref_data = loadmat('../ref/ldc_Re100.mat')
refu = ref_data['u'].T[...,jnp.newaxis]
refv = ref_data['v'].T[...,jnp.newaxis]
refx = ref_data['x'].squeeze()
refy = ref_data['y'].squeeze()
# x,y = jnp.meshgrid(x,y)
# x = jnp.expand_dims(x.flatten(),axis=-1)
# y = jnp.expand_dims(y.flatten(),axis=-1)
# xy = jnp.concatenate((x,y),axis=-1)
sol = jnp.concatenate((refu,refv),axis=-1)
solx = sol[128,:,1]
soly = sol[:,128,0]

In [ ]:
x = np.linspace(0, 2, 1000)
y = np.linspace(0, 2, 1000)
outx = interp(x,np.ones_like(x))[...,1]/U_lid
outy = interp(np.ones_like(y),y)[...,0]/U_lid

In [ ]:
plt.plot(refy,solx)
plt.plot(x/2,outx)
plt.legend(['Reference','Simulation'])
plt.show()

In [ ]:
plt.plot(refy,soly)
plt.plot(y/2,outy)
plt.legend(['Reference','Simulation'])
plt.show()

# Tri mesh test


In [ ]:
from scipy.io import loadmat
data = loadmat("IRTmesh.mat")

In [ ]:
mesher = Mesher()
mesher.points = np.array([data["NODE"][0,i][2][0][:,0] for i in range(data["NODE"].shape[1])], dtype=np.float64)
mesher.cells = np.array([[data["CELL"][0,i][j][0][0][0] for j in range(6,9)] for i in range(data["CELL"].shape[1])],dtype=np.int32)-1
mesher.faces = np.array([[data["FACE"][0,i][j][0][0][0] for j in range(7,9)] for i in range(data["FACE"].shape[1])],dtype=np.int32)-1
mesher.point_markers = np.zeros_like(mesher.points[:,0])

mesher.points[19800]=np.asarray([1.992,1.992])

mesher.enforce_ccw()

print(mesher.points.shape, mesher.cells.shape, mesher.faces.shape, mesher.point_markers.shape)

mesher.calc_mesh_properties()

In [ ]:
mesher.points[:,1]

In [ ]:
mesher.points[:,0] == jnp.asarray(0.02)

In [ ]:
for i in jnp.where(mesher.points[:,1] == jnp.asarray(1.99))[0]:
    for j in jnp.where(mesher.points[:,0] == jnp.asarray(1.99))[0]:
        if i==j:
            print(f"i={i}, j={j}, point={mesher.points[i]}")

In [ ]:
mesher.points[19800].at.set([1.992,1.992])

In [ ]:
mesher.verify_stencil_geometry()

In [ ]:
# Given physical parameters
Re = 100            # Reynolds number
nu = 0.1     # physical kinematic viscosity
L = 100             # domain length (physical units)
dx = 1           # spatial step (physical units)
dt = 0.1            # time step (physical units)

# Speed of sound squared for D2Q9 lattice
c_s_sq = 1.0 / 3.0

# Convert to lattice units
U_lattice = Re * nu / L

# Compute relaxation time tau
Tau = nu / c_s_sq + 0.5

# Print results
print(f"Lid velocity (lattice units): {U_lattice:.5f}")
print(f"Relaxation time tau: {Tau:.5f}")
U_lid = U_lattice
dynamics = D2Q9(tau=Tau,delta_t = dt)

In [ ]:
cells,faces,nodes = mesher.to_env(dynamics,flux_method="cc_lax_wendroff")
env = Environment(cells,faces,nodes)
env.init()

In [ ]:
for i in tqdm(range(100000)):
    env = env.step()

In [ ]:
env = env.step()

In [ ]:
env

In [ ]:
xy = np.array([data["CELL"][0,i][4][0][:,0] for i in range(data["CELL"].size)])
x = xy[:,0]
y = xy[:,1]
vel = env.cells.vel 
mag = np.sqrt(np.sum(vel**2,axis=-1))
plt.quiver(x,y,vel[:,0],vel[:,1],scale=1)
plt.axis([0,.1,0,.1])
xy = np.asarray([data["NODE"][0,i][2,0][:,0] for i in range(data["NODE"].size)])
x = xy[:,0]
y = xy[:,1]
tri = np.asarray([data["CELL"][0,i][6:9][:,0] for i in range(data["CELL"].size)])-1
plt.triplot(x,y,tri)
plt.axis([1.9,2,1.9,2])
plt.show()

In [ ]:
xy = np.array([data["CELL"][0,i][4][0][:,0] for i in range(data["CELL"].size)])
x = xy[:,0]
y = xy[:,1]
vel = env.cells.vel
mag = np.sqrt(np.sum(vel**2,axis=-1))
plt.scatter(x,y,c=mag,cmap='viridis',vmin=0,vmax=U_lid,s=.5)
plt.axis('equal')
plt.show()